In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import mediapipe as mp     # 載入 mediapipe 函式庫
import time
import os
import json
import urllib.request
from urllib.parse import urlparse
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import warnings


In [2]:
chrome_options = Options()
# 指定下載資料夾的路徑
download_folder = r"C:\Users\user\Downloads\hammer_ori\dir"
# 添加配置參數，設置默認下載路徑
prefs = {"download.default_directory" : download_folder}
chrome_options.add_experimental_option("prefs", prefs)

In [3]:

driver = webdriver.Chrome(options=chrome_options)

The chromedriver version (123.0.6312.86) detected in PATH at c:\Users\user\Desktop\behance-download\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.119); currently, chromedriver 124.0.6367.155 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


In [31]:
import requests  # 導入requests模組


#==================從person_page取得圖片連結======================================
def filter_links(links, current_url):
    new_links = []
    for link in links:
        if("game-design/game-design" in link):
            continue
        if(link in new_links):
            continue
        if(link == current_url):
            continue
        new_links.append(link)
    return new_links



def get_links_from_person_page(driver):
    link_tags = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//article"))) #搜索所有<a>標籤

    all_links = []
    for link_tag in link_tags:
        a_tags = link_tag.find_elements(By.XPATH, ".//a")
        links = [tag.get_attribute("href") for tag in a_tags]
        links = filter_links(links, driver.current_url)
        
        if(len(links) > 1):
            # warnings.warn("muti link tags found in pin page, only first one will be used")
            raise Exception("muti link tags found in pin page, only first one will be used", links)
            
        all_links.append(links[0])
    return all_links


In [32]:

#==================從gallery取得圖片連結======================================
def get_gallery_links(driver):
    sections = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//section"))) #搜索所有<sections>標籤
    print("sections", sections)
    link_list = []
    for section in sections:
        imgs = section.find_elements(By.XPATH, ".//img")
        
        if(len(imgs) > 1):
            warnings.warn("muti link tags found in pin page, only first one will be used")
            print("muti link tags found in pin page, only first one will be used", imgs)
        if(len(imgs) == 0):
            print("no img found in section")
            continue
            
        link = imgs[0].get_attribute("src")
        link_list.append(link)
    return link_list
def download_image(image_url, filename):
    try:
        response = requests.get(image_url)  # 向圖片的URL發送GET請求
        response.raise_for_status()  # 檢查請求是否成功

        with open(filename, 'wb') as f:  # 打開一個新文件（二進位模式）
            f.write(response.content)  # 將圖片內容寫入文件
        print("Image downloaded successfully: {}".format(filename))
    except requests.RequestException as e:
        print("Failed to download the image: {}".format(e))

def download_images_from_gallery_page(driver, index, download_folder="download_folder"):
    print("download_images_from_gallery_page")
    links = get_gallery_links(driver)
    print(links)
    if(not os.path.exists(download_folder)):
        os.makedirs(download_folder)

    for i, link in enumerate(links):
        doanload_path = os.path.join(download_folder, f"image_{index}_{i}.jpg")
        download_image(link, doanload_path)
        time.sleep(1)
# get_gallery_links(driver)

In [33]:
#==================取得一頁的連結有關的function======================================
def get_page_link(driver):
    page_link = []
    while(1): #持續下滑頁面直到沒有新連結
        new_link = get_more_link(driver, page_link)
        page_link += new_link
        if(len(new_link) == 0):
            break
    return page_link

def get_more_link(driver, page_link, sleep_time=2, repeat_time=5):        
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) #滾動頁面以取得更多連結
    time.sleep(sleep_time)

    page_links = get_links_from_person_page(driver)
    new_links = [link for link in page_links if link not in page_link] #取得新連結
    if(len(new_links) == 0): 
        if(repeat_time > 0): #如果沒有新連結，則再次下滑頁面，最多重複repeat_time次
            print("no new link, try PAGE_DOWN again")
            new_links = get_more_link(driver, page_link, sleep_time=sleep_time, repeat_time=repeat_time-1)
        else: 
            return []
    return new_links
def change_page(driver, link, sleep_time=1):
    print("change_page to: ", link)
    driver.get(link)
    time.sleep(sleep_time)


In [36]:
def download_images_from_person_page(driver, download_folder="download_folder"):
    print("download_images_from_person_page")
    gallery_links = get_page_link(driver) #取得當前頁面的所有連結
    print("gallery_links", gallery_links)
    if(not os.path.exists(download_folder)):
        os.makedirs(download_folder)

    for i, gallery_link in enumerate(gallery_links):
        change_page(driver, gallery_link) #切換到pin_page
        download_images_from_gallery_page(driver, i, download_folder=download_folder)
        time.sleep(1)


person_id = "omanmarina02a5"
driver.get(f'https://www.behance.net/{person_id}')
download_folder = os.path.join("download_folder", person_id)
download_images_from_person_page(driver, download_folder=download_folder)

download_images_from_person_page
no new link, try PAGE_DOWN again
no new link, try PAGE_DOWN again
no new link, try PAGE_DOWN again
no new link, try PAGE_DOWN again
no new link, try PAGE_DOWN again
gallery_links ['https://www.behance.net/gallery/165039121/slots', 'https://www.behance.net/gallery/165039009/slots', 'https://www.behance.net/gallery/165038907/slots', 'https://www.behance.net/gallery/165038747/slots', 'https://www.behance.net/gallery/165029481/slots', 'https://www.behance.net/gallery/165024605/slots', 'https://www.behance.net/gallery/165024649/slots']
change_page to:  https://www.behance.net/gallery/165039121/slots
download_images_from_gallery_page
sections [<selenium.webdriver.remote.webelement.WebElement (session="7ff271ac92f3e04ac425e00990922318", element="f.47C986A33EECD96799A068D26CBCAA9C.d.650C46139AB05F527736162F79A202DE.e.31536")>]
['https://mir-s3-cdn-cf.behance.net/project_modules/1400/11d8b6165039121.640097db1e294.jpg']
Image downloaded successfully: download_fol